# PaddleOCR

In [ ]:
# ignore if running outside Google Colab
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/TraditionalMedicineChatbot/

In [ ]:
!pip install paddlepaddle paddlepaddle-gpu
!pip install paddleocr
!pip install "langchain==0.0.353"

In [ ]:
# --- PaddleOCR Initialization ---
from paddleocr import PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang='vi')

In [ ]:
# --- OCR Usage: Process Image ---
import os
img_filename = 'sample.png'
img_path = os.path.abspath(img_filename) if os.path.exists(img_filename) else None
if img_path is None:
    raise FileNotFoundError("Could not find 'sample.png' locally. Upload it or mount Google Drive and set img_path accordingly.")
print(f'Using image: {img_path}')
try:
    result = ocr.predict(img_path)
except Exception:
    result = ocr.ocr(img_path, cls=True)

In [ ]:
import json
import numpy as np

# 1. Prepare data container
output_data = []
ocr_data = result[0]

rec_texts = ocr_data.get('rec_texts', [])
rec_scores = ocr_data.get('rec_scores', [])
rec_polys = ocr_data.get('rec_polys', [])

# 2. Convert data to standard Python types (to avoid JSON errors)
if rec_texts:
    for i in range(len(rec_texts)):
        # Handle the box: Convert numpy array to list if necessary
        box = rec_polys[i]
        if isinstance(box, np.ndarray):
            box = box.tolist()

        # Handle the score: Convert numpy float to python float
        score = rec_scores[i]
        if isinstance(score, (np.float32, np.float64)):
            score = float(score)

        # Create a structured dictionary for this detection
        detection = {
            "id": i + 1,
            "text": rec_texts[i],
            "confidence": score,
            "box": box
        }
        output_data.append(detection)

# 3. Write to JSON file
output_filename = 'ocr_result.json'
try:
    with open(output_filename, 'w', encoding='utf-8') as f:
        json.dump(output_data, f, ensure_ascii=False, indent=4)
    print(f"Successfully saved {len(output_data)} detections to '{output_filename}'")
except Exception as e:
    print(f"Error saving JSON: {e}")

# Chandra

In [ ]:
%pip install chandra-ocr

In [ ]:
from chandra_ocr import ChandraOCR


# Initialize the OCR engine 
ocr = ChandraOCR(lang='vi')  

pdf_filename = 'sample.pdf'
pdf_path = os.path.abspath(pdf_filename) if os.path.exists(pdf_filename) else None

# Run OCR on the PDF
results = ocr.ocr_pdf(pdf_path)

# Each result contains text and bounding box info for each page
for page_num, page in enumerate(results, 1):
    print(f"Page {page_num}:")
    for block in page:
        print(block['text'], block['box'])